## Word2Vec 알고리즘


In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
path = '../input/t-academy-recommendation2/movies/'

In [4]:
movie = pd.read_csv(path + 'ratings.csv', low_memory=False)
movie.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [5]:
movie = movie.sort_values(by='timestamp', ascending=True).reset_index(drop=True)
movie.head()

,userId,movieId,rating,timestamp
0,383,21,3.0,789652009
1,383,47,5.0,789652009
2,383,1079,3.0,789652009
3,409,21,5.0,828212412
4,409,25,4.0,828212412


In [6]:
# 영화의 Metadata를 불러와서 movieID에 맞는 TITLE을 구해줍니다. 
meta = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [7]:
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [8]:
meta = meta.rename(columns={'id':'movieId'})
movie['movieId'] = movie['movieId'].astype(str)
meta['movieId'] = meta['movieId'].astype(str)

movie = pd.merge(movie, meta[['movieId', 'original_title']], how='left', on='movieId')

In [9]:
movie = movie[movie['original_title'].notnull()].reset_index(drop=True)

In [10]:
agg = movie.groupby(['userId'])['original_title'].agg({'unique'})
agg.head()

,unique
userId,
1,"[Jay and Silent Bob Strike Back, Vivement dima..."
2,"[Terminator 3: Rise of the Machines, The Conve..."
3,"[300, The Killing, Shortbus, Finding Neverland..."
4,"[David, The Wedding Planner, Casablanca, Sleep..."
5,"[Gleaming the Cube, Cool Hand Luke, Hidalgo, U..."


In [11]:
movie['original_title'].unique()

array(['The Endless Summer', 'Jarhead', '彼女の想いで', ...,
       'The Lonedale Operator', 'Violeta se fue a los cielos',
       'To Kill a Priest'], dtype=object)

Word2vec 적용

In [12]:
# int형식은 Word2vec에서 학습이 안되어서 String으로 변경해줍니다. 
sentence = []
for user_sentence in agg['unique'].values:
    sentence.append(list(map(str, user_sentence)))

In [13]:
# Word2vec의 학습을 진행해줍니다. 
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence, size=20, window = 5, 
                           min_count=1, workers=4, iter=200, sg=1)

In [14]:
embedding_model.wv.most_similar(positive=['Spider-Man 2'], topn=10)

[('Snow Cake', 0.8505643606185913),
 ('Domicile Conjugal', 0.7630471587181091),
 ('Licence to Kill', 0.7270383834838867),
 ('I, Robot', 0.7246425151824951),
 ('The Godfather', 0.7142737507820129),
 ('Krull', 0.7069061994552612),
 ('Forrest Gump', 0.7063434720039368),
 ('Some Like It Hot', 0.7042686939239502),
 ('Heavenly Creatures', 0.7001181840896606),
 ('Star Trek: Nemesis', 0.6986316442489624)]

## Doc2Vec 적용

![](https://drive.google.com/uc?export=view&id=1g2ausKfoaAT0jMwSatRUG3fiGWfDuysV
)

In [15]:
from gensim.models import doc2vec

In [16]:
meta = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
meta = meta[meta['original_title'].notnull()].reset_index(drop=True)
meta = meta[meta['overview'].notnull()].reset_index(drop=True)

In [17]:
from nltk.corpus import stopwords 
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
import re 
stop_words = set(stopwords.words('english')) 

overview = []
for words in tqdm(meta['overview']):
    word_tokens = word_tokenize(words)
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    sentence = sentence.strip()
    
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    overview.append(result)

In [18]:
meta['pre_overview'] = overview

In [19]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=0,            # PV-DBOW / default 1
    dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    size=100,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    workers=4,   # multi cpu
    hs = 1,          # hierar chical softmax / default 0
    negative = 10   # negative sampling / default 5
)

In [20]:
from collections import namedtuple

agg = meta[['id', 'original_title', 'pre_overview']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['original_title', 'pre_overview']].values]

In [21]:
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dbow+w,d100,n10,hs,w10,mc5,s0.001,t4)


In [22]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(5)):
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

#doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
end = time()
print("During Time: {}".format(end-start))


During Time: 766.1116979122162


In [23]:
doc_vectorizer.docvecs.most_similar('Toy Story', topn=20)

[('Letzte Worte', 0.7423182725906372),
 ('La moutarde me monte au nez', 0.6931746006011963),
 ('Kader', 0.6900465488433838),
 ('Children in the Surf at Coney Island', 0.6730849146842957),
 ('Przechodzien', 0.6717959642410278),
 ('It Stains the Sands Red', 0.6665500402450562),
 ('Stryapukha', 0.6633526086807251),
 ('Unstrung Heroes', 0.6624222993850708),
 ('По следам бременских музыкантов', 0.6539246439933777),
 ('Особенности национальной политики', 0.6518951654434204),
 ('The Aristocats', 0.6504817008972168),
 ('Skazka o Poteryannom Vremeni', 0.6501118540763855),
 ('12 Feet Deep', 0.6472882032394409),
 ('El vendedor de humo', 0.6385210156440735),
 ('My Friends Need Killing', 0.6351690292358398),
 ('Water & Power: A California Heist', 0.6296603083610535),
 ('Thunder Road', 0.6233446598052979),
 ('Due Amici', 0.6210700869560242),
 ('El Gran Vázquez', 0.6210695505142212),
 ('Kaos', 0.6188639998435974)]

In [24]:
doc_vectorizer.docvecs.most_similar('Harry Potter and the Deathly Hallows: Part 1', topn=20)

[('Skidoo', 0.7517136931419373),
 ('Demonic Toys', 0.7213362455368042),
 ('The Great Ecstasy of Robert Carmichael', 0.7018694877624512),
 ('$ Dollars', 0.7014257311820984),
 ('Kasaba', 0.6635128259658813),
 ('Se sei vivo spara', 0.6613894104957581),
 ('Just Like Us', 0.6587302088737488),
 ('The Blacksmith', 0.6484951376914978),
 ('1990: I guerrieri del Bronx', 0.648390531539917),
 ('Blood River', 0.6459505558013916),
 ('Handsome Harry', 0.6454256772994995),
 ('Der Räuber', 0.6429744958877563),
 ('Une femme douce', 0.641798734664917),
 ('কাঞ্চনজঙ্ঘা', 0.6296833753585815),
 ('Saint John of Las Vegas', 0.6182495355606079),
 ('Glue', 0.6150935292243958),
 ('20 Seconds of Joy', 0.6144537329673767),
 ('Il deserto dei Tartari', 0.6135669350624084),
 ('Dragonlance: Dragons Of Autumn Twilight', 0.6117035746574402),
 ('The Prizefighter and the Lady', 0.6052166223526001)]